## Getting a bit more deeper into Agents

### Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [5]:
from langchain.agents import create_react_agent, load_tools, initialize_agent
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

In [7]:
tools = load_tools(["llm-math"],  llm=llm)

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,)

/home/miller/.local/share/virtualenvs/langchain_open_rag-FDDFjH_q/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [9]:
output_1=conversational_agent.run("Add 7 to 9 and tell me the result")
output_2=conversational_agent.run("Add 5 to the previous result")

/home/miller/.local/share/virtualenvs/langchain_open_rag-FDDFjH_q/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 7 to 9
Observation: Answer: 16
Thought:Do I need to use a tool? No
AI: The result of adding 7 to 9 is 16.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 16 to 5
Observation: Answer: 21
Thought:Do I need to use a tool? No
AI: The result of adding 5 to the previous result of 16 is 21.

> Finished chain.


In [11]:
print(output_1)
print(output_2)

The result of adding 7 to 9 is 16.
The result of adding 5 to the previous result of 16 is 21.


## ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [18]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer
from langchain_openai import OpenAI

In [19]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

#intializing the agent
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8
)

In [27]:
docstore_agent("Tell me a few key things about Mahatma Gandhi.")



> Entering new AgentExecutor chain...
Thought: I need to search Mahatma Gandhi and find key information about him.
Action: Search[Mahatma Gandhi]

KeyboardInterrupt: 

## Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.

In [23]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [28]:
print(agent.invoke("Question: Who was the president of USA when the first Moom landing took place?"))



> Entering new AgentExecutor chain...
 Yes.
Follow up: When did the first Moon landing take place?

RatelimitException: https://duckduckgo.com/ 202 Ratelimit

The above code imports necessary modules from the LangChain library, including agents and language models. It sets up a conversational agent with a specific agent configuration called “self-ask-with-search”. The agent utilizes the “Intermediate Answer” tool, which performs Wikipedia searches.